## Purpose of script:
#### Reviewing Spark's recommender system implementation
#### Referencing Jose Portilla's "Spark and Python for Big Data with PySpark" course

In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
data = spark.read.csv('../Datasets/movielens_ratings.csv', inferSchema=True, header=True)

In [4]:
data.show(5)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
+-------+------+------+
only showing top 5 rows



In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
training_data, test_data = data.randomSplit([0.8, 0.2])

In [8]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [9]:
model = als.fit(training_data)

In [10]:
 preds = model.transform(test_data)

In [11]:
preds.show(5)

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      1|   1.0|    28| 1.1710202|
|      1|   1.0|    26|-1.3281746|
|      0|   1.0|     6|0.36570388|
|      0|   1.0|    15| 0.8831788|
|      0|   1.0|     8|0.82339805|
+-------+------+------+----------+
only showing top 5 rows



In [12]:
test_data.count()

291

In [13]:
preds.count()

291

In [14]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [15]:
# this is a bad prediction as it is out of possible values of 1 - 5,
# however it is likely due to how small the sample dataset is
rmse = evaluator.evaluate(preds)
print(f'Root Mean Square Error: {rmse}')

Root Mean Square Error: 1.6592627955269772


In [17]:
single_user = test_data.filter(test_data['UserId'] == 11).select(['movieId', 'userId'])

In [18]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|     16|    11|
|     21|    11|
|     25|    11|
|     40|    11|
|     45|    11|
|     50|    11|
|     61|    11|
|     66|    11|
|     94|    11|
+-------+------+



In [19]:
recommendations = model.transform(single_user)

In [20]:
# creating predictions / recommendations for a single user
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     94|    11| 5.3945665|
|     50|    11|  2.857252|
|     40|    11| 2.6870677|
|     21|    11| 2.4183254|
|      6|    11| 2.3852937|
|     16|    11| 1.5475124|
|     61|    11| 1.4599025|
|     25|    11| 1.0896378|
|     45|    11|0.74331963|
|     66|    11| 0.6996665|
+-------+------+----------+



In [21]:
spark.stop()